inspiration: 
    
http://toddwschneider.com/posts/chicago-taxi-data/

https://github.com/toddwschneider/chicago-taxi-data

I: PostgreSQL + EDA

    general trend

   **driver behavior**
    
   **R code** 
   
   **Kaggle**
    
II: Taxi / bike / public transit

III: Taxi Pricing

   **ML models**
   
IV: Trip Duration
    
V: LDA

    daily pattern 
            

## Table of Content
* Data Source
* Built a PostgreSQL Database
    * create a table
* general trends
    * trip volume
    * trip revenue
* community areas
* active taxis
* taxi company
* Pricing

## Data Source

[City of Chicago](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew)  

https://digital.cityofchicago.org/index.php/chicago-taxi-data-released/
a total of 112,860,054 records
columns

community map dowloaded at [here](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Community-Areas-current-/cauq-8yn6)

Download data through [Socrata](https://dev.socrata.com/foundry/data.cityofchicago.org/wrvz-psew) API
vDnOwjU97M5L6B7hHD7A2cgqL

In [2]:
# from sodapy import Socrata
import pandas as pd
import numpy as np
import json
import time
from datetime import timedelta

import os
# multiprocessing
from multiprocessing.pool import Pool

In [3]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", "vDnOwjU97M5L6B7hHD7A2cgqL")

In [4]:
_src_dir_ = "/Users/wding/Documents/taxi_data/"

In [5]:
def get_next_day(start):
    next_day = (pd.to_datetime(start) + timedelta(days=1)).strftime("%Y-%m-%d")
    return next_day

In [6]:
def get_all_days(start):
    day_list = []
    for i in range(1673):
        next_day = (pd.to_datetime(start) + timedelta(days=1)).strftime("%Y-%m-%d")
        day_list.append(start)
        start = next_day
    return day_list

In [7]:
def download_data(start):
    next_day = get_next_day(start)
    condition = """trip_start_timestamp >= '%sT00:00:00' and 
                trip_start_timestamp < '%sT00:00:00'""" %(start,next_day)
    # Results returned as JSON from API / converted to Python list of
    # dictionaries by sodapy.
    temp = client.get("wrvz-psew", 
                      where=condition,
                      limit=500000)
    # Convert to pandas DataFrame
    df = pd.DataFrame.from_records(temp)
    df.to_csv(_src_dir_ + start + ".csv", index=False)

In [ ]:
### Download Files

In [8]:
day_list = get_all_days("2013-01-01")
start_time = time.time()
pool = Pool(os.cpu_count() - 1)
pool.map(download_data, day_list)
pool.close()
pool.join()
print(round((time.time() - start_time)/60,2), " min to download 1673 files")

80.1  min to download 1673 files


## Built a postgreSQL database

In [3]:
import psycopg2

/Users/wding/anaconda3/envs/py3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
conn = psycopg2.connect("dbname=chicago_taxi user=postgres host=localhost password=wzd")
cur = conn.cursor()

### create a table

In [15]:
cur.execute("""CREATE TABLE taxi_trips(
            company text, 
            dropoff_census_tract float8  ,
            dropoff_centroid_latitude float8  ,
            dropoff_centroid_location text,
            dropoff_centroid_longitude float8,
            dropoff_community_area int,
            extras float8,
            fare float8,
            payment_type char(15),
            pickup_census_tract float8,
            pickup_centroid_latitude float8,
            pickup_centroid_location text,
            pickup_centroid_longitude float8,
            pickup_community_area int,
            taxi_id text,
            tips float8,
            tolls float8,
            trip_end_timestamp timestamp,
            trip_id text,
            trip_miles float8, 
            trip_seconds int,
            trip_start_timestamp timestamp,
            trip_total float8,
            PRIMARY KEY(taxi_id, trip_id)
            )""")
conn.commit()

In [ ]:
### Copy all files to table

In [10]:
def csv_to_table(file_list):
    for file in file_list:
        _dir = _src_dir_ + file
#         print(file)
        cur.execute("""COPY taxi_trips FROM '%s' DELIMITER ',' CSV HEADER;""" %_dir) 
        conn.commit() 

In [42]:
conn.close()

In [87]:
conn.commit()

In [ ]:
start_time = time.time()
file_list = [f for f in os.listdir(_src_dir_) if f.endswith(".csv")]
csv_to_table(file_list)

In [19]:
print("finished loading all .csv file to database")

finished loading all .csv file to database


Check the number of records in the table.

In [20]:
query = """select count(*) from taxi_trips;"""
cur.execute(query)
rows = cur.fetchall()
print("Total Number of Records: ", rows[0][0])

Total Number of Records:  112859983


## General Trends

### Trip Volume

The data makes clear that taxi usage in Chicago has declined dramatically since 2014. Chicago taxi usage was declining at an average 2.54% monthly rate since peaking in May 2014. Also, the annual St. Patrick’s Day Parade turned out to be the day with the most taxi trips in Chicago every year since 2013.

In [6]:
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [22]:
def time_series_plot(df, col1, col2, color, title):
    trace = go.Scatter(x=df[col1],
                       y=df[col2],
                       line=dict(color=color))
    data = [trace]
    layout = dict(
        title=title,
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1 month',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6 month',
                         step='month',
                         stepmode='backward'),
                     dict(count=1,
                        label='1 year',
                        step='year',
                        stepmode='backward'),
                    dict(count=1,
                        label='YTD',
                        step='year',
                        stepmode='todate'),
                    dict(step='all')
                ])
            ),
            rangeslider=dict(),
            type='date'
        )
    )

    fig = dict(data=data, layout=layout)
    iplot(fig)

In [116]:
query = """select distinct date_trunc('day', trip_start_timestamp), count(trip_start_timestamp) 
        from taxi_trips group by date_trunc('day', trip_start_timestamp);"""
cur.execute(query)
rows = cur.fetchall()
day_df = pd.DataFrame.from_records(rows)
day_df.columns = ["day", "count"]

In [25]:
time_series_plot(day_df, "day", "count", '#1f77b4', "Time Series of Trip Volume")

In [122]:
day_df["year"] = day_df["day"].apply(lambda x : str(x)[:4])
day_df["month"] = day_df["day"].apply(lambda x : str(x)[5:7])

In [160]:
month = day_df.groupby(["year","month"])["count"].sum().reset_index()
month["last_month_count"] = month["count"].shift(1)
month["change"] = (month["count"] - month["last_month_count"])/month["last_month_count"] 
print("Maximum trip volume happened at: ")
print(month[month["count"] == max(month["count"])])
month_2014_6 = month[17:]
print("Monthly decrease rate since peak month: %.3f" %(
    np.mean(month_2014_6["change"])*float(100)) + "%")

Maximum trip volume happened at: 
    year month    count  last_month_count    change
16  2014    05  2883632         2653233.0  0.086837
Monthly decrease rate since peak month: -2.541%


### Trip Revenue 

Revenue generated by taxi trips has decreased as well but its peak values occured in 2015 and it took a while for the decrease in trip volume reflect that in trip revenue.

In [35]:
query = """select distinct date_trunc('day', trip_start_timestamp), sum(trip_total) 
from taxi_trips group by date_trunc('day', trip_start_timestamp);"""
cur.execute(query)
rows = cur.fetchall()
fare_df = pd.DataFrame.from_records(rows)
fare_df.columns = ["day", "trip_total"]

In [36]:
time_series_plot(fare_df, "day", "trip_total", '#F06A6A' ,"Time Series of Trip Fare")

### Active Taxi

The number of active taxi per day was stable till the end of 2015. 

In [41]:
query = """select distinct date_trunc('day', trip_start_timestamp), count(distinct taxi_id) 
from taxi_trips where trip_seconds > 0 group by date_trunc('day', trip_start_timestamp) ;"""
cur.execute(query)
rows = cur.fetchall()
df = pd.DataFrame.from_records(rows)
df.columns = ["day", "active_taxi"]
time_series_plot(df, "day", "active_taxi", '#33CFA5' ,"Time Series of Active Taxis")

## Geolocation

We use Bokeh to create interactive maps to visualize how trip fair changed on each community area over time.

https://data-dive.com/cologne-bike-rentals-interactive-map-bokeh-dynamic-choropleth

In [8]:
query = """select EXTRACT(YEAR from taxi_trips.trip_start_timestamp),
taxi_trips.pickup_community_area, 
avg(taxi_trips.fare)
from taxi_trips where pickup_community_area is not null and fare > 0 
group by taxi_trips.pickup_community_area,
EXTRACT(YEAR from taxi_trips.trip_start_timestamp)"""
cur.execute(query)
rows = cur.fetchall()
df = pd.DataFrame.from_records(rows)
df.columns = ["year", "com_area", "fare_avg"]

In [1]:
import fiona
from bokeh.io import show, output_notebook
output_notebook() 

from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure, save
from bokeh.layouts import column, row, widgetbox
from bokeh.models import CustomJS, Slider, Toggle, mappers, sources, tools
palette.reverse()

Loading BokehJS ...

In [9]:
SHAPEFILE="data/geo_export_06ed62a0-9a5d-4474-b0d1-b59105ec7708.shp" # community area
shp = fiona.open(SHAPEFILE)

In [10]:
district_name = [ feat["properties"]["area_numbe"] for feat in shp ]
district_x = [ [x[0] for x in feat["geometry"]["coordinates"][0]] for feat in shp]
district_y = [ [x[1] for x in feat["geometry"]["coordinates"][0]] for feat in shp]
color_mapper = mappers.LogColorMapper(palette=palette)

In [11]:
temp = pd.DataFrame([int(i) for i in district_name])
temp.columns = ["com_area"]
# merge by community area
match_df = temp.merge(df, on ="com_area", how = "left")

In [12]:
rates = []
for i in range(2013, 2018):
    rates.append(list(match_df[match_df["year"] == i]["fare_avg"]))
# from list to dict 
rate_dict = {str(i): v for i, v in enumerate(rates)} 
data = dict(x=district_x, y=district_y, name=district_name,
            rate=rates[0], **rate_dict)
source = sources.ColumnDataSource(data) 

In [13]:
TOOLS = "pan,wheel_zoom,reset,hover,save"
p = figure(
    title="Change of Average Taxi Fare Over Time", tools=TOOLS,
    x_axis_location=None, y_axis_location=None
)
p.grid.grid_line_color = None
p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

hover = p.select_one(tools.HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Census Tract", "@name"),
    ("Average Taxi Fare", "@rate"),
    ("(Long, Lat)", "($x, $y)"),
]

# show(p)

In [14]:
# add slider with callback to update data source
slider = Slider(start=0, end=4, value=0, step=1, title="Year")

def update(source=source, slider=slider, window=None):
    """ Update the map: change the density measure according to slider
        will be translated to JavaScript and Called in Browser """
    data = source.get('data')
    v = cb_obj.get('value')
    data['rate'] = data[str(v)]
    source.trigger('change')

slider = Slider(start=0, end=4, value=1, step=1, title="Year",
            callback=CustomJS.from_py_func(callback))
slider.js_on_change('value', CustomJS.from_py_func(update))
show(column(p,widgetbox(slider),))

## Taxi Drivers' Performances

Number of trips per taxi per day

Average number of work hours per taxi per day

## Taxi Company

all company select trip_total, fare, volume 

## Type of Payment 

In [ ]:
select by company

## Pricing ($/hour)

pricing by different company

In [200]:
fare = fetchdata[fetchdata.trip_miles > 0]
fare['fare_mile'] = fare.fare / fare.trip_miles

D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [ ]:
histogram 

## Daily Change

rush hour, commute & entertainment 

In [ ]:
## average number of trips per hour

In [ ]:
average duration of a taxi trip